# Summarization of news articles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/'Colab Notebooks'/capstone_project

/content/drive/MyDrive/Colab Notebooks/capstone_project


In [ ]:
!pip install transformers[torch] datasets
!pip install evaluate
!pip install rouge_score

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
import evaluate
import torch
# import nltk
import pandas as pd

In [ ]:
# nltk.download('punkt')

In [ ]:
model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used

In [ ]:
df = pd.read_csv("ikonrecent.csv")

In [ ]:
df

,Unnamed: 0,article url,title,article
0,0,https://ikon.mn/n/354p,Өнгөрсөн сард улсын хэмжээнд 10.5 сая тонн ача...,Зам тээврийн салбар энэ оны дөрөвдүгээр сард у...
1,1,https://ikon.mn/n/354o,Шатахуун түгээхийн болон түргэн тусламжийн маш...,Хан-Уул дүүргийн нутаг дэвсгэрт шатахуун түгээ...
2,2,https://ikon.mn/n/354k,Монголын хөлбөмбөгийн шигшээ баг Азийн АШТ-ий ...,2027 онд Саудын Араб улсад зохион байгуулагдах...
3,3,https://ikon.mn/n/354m,Цэцэг төвийн уулзварын зам засварын ажил марга...,"Цэцэг төвийн уулзвараас Цаг уур, орчны шинжилг..."
4,4,https://ikon.mn/n/354h,"""Монголбанкны эх үүсвэрээр орон сууц авч буй и...",Ипотекийн зээлийн үндсэн шалгуур болон зээл ол...
5,5,https://ikon.mn/n/354e,"""207-р байрны галд өртсөн найман хувийг засуул...",Газын дэлбэрэлтэд өртсөн 207 дугаар байрны орш...
6,6,https://ikon.mn/n/354l,“Grow with Google Mongolia” хөтөлбөрийн дэмжиг...,Монгол Улсад анх удаа хэрэгжүүлж буй “Grow wit...
7,7,https://ikon.mn/n/354j,"Хуримтлалын сангийн дансанд өнөөдөр Эрдэнэт, О...",Үндэсний баялгийн сангийн тухай хууль хэрэгжиж...
8,8,https://ikon.mn/n/354i,Буудлагын Дэлхийн цомын тэмцээнд О.Есүгэн найм...,Азербайжаны нийслэл Баку хотноо буудлагын Дэлх...
9,9,https://ikon.mn/n/354g,Бороотой үеэр Сүхбаатар дүүргийн ногоон байгуу...,Өнөөдөр өглөө бороотой байсан ч Сүхбаатар дүүр...


## Inference Example

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_path = "ikon_summarization"
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)

In [ ]:
def generate_summary(text):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)

    return summary

In [ ]:
df['article'][5]

'Газын дэлбэрэлтэд өртсөн 207 дугаар байрны оршин суугчид одоогийн нөхцөл байдал болон төрөөс ямар арга хэмжээ авч байгаа талаар өнөөдөр мэдээлэл хийлээ.2024 оны нэгдүгээр сарын 24-ний\xa0өдөр "Дашваанжил" ХХК-ийн газ тээвэрлэж явсан машин, Патрол\xa0загварын\xa0машин Дүнжингаравын урд уулзвар дээр осол гаргасны улмаас галд 207 дугаар байрны нийт 63 өрхийн 218 оршин суугч\xa0өртөж орон гэргүй болсон юм.Ослын улмаас тус байранд амьдарч байсанБарилга хот байгуулалтын яамны барилгын дүгнэлттэй оршин суугчид тавдугаар сарын 1-нд танилцсан бөгөөд уг мэдээлэл нь Улсын Онцгой Комиссын дөрөвдүгээр сарын 4-нд хийсэн хэвлэлийн бага хурлаас зөрүүтэй байна.Оршин суугчдад мэдэгдэлгүйгээр хийсэн УОК-ын тус мэдэгдэл хэтэрхий нэг талыг барьсан сөрөг агуулгатай, дүгнэлтийн тоон дүнгүүдээс зөрсөн, ташаа мэдээлэл тараасан гэж үзэж байгаа\xa0гэдгийг оршин суугчид онцлов.Тус дүгнэлтийн талаар207-р байрны оршин суугч"УОК-оос нийт барилгын 86% бүрэн шатсан гэж мэдээлэл өгсөн. Гэтэл бодит дүгнэлт нь дээр нийт

In [ ]:
generate_summary(df['article'][5])

'"Дүнжингаравын урд уулзвар дээр ослын улмаас галд өртсөн найман хувийг л засуулаад байрандаа орох боломжийг л төр засагт хүргэмээр байна"'